In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Subtract
from tensorflow.keras.optimizers import Adam

# Define the base network
def create_base_network(input_dim):
    input = Input(shape=(input_dim,))
    x = Dense(64, activation='relu')(input)
    x = Dense(32, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    output = Dense(1, activation='linear')(x)  # Output layer for regression
    return Model(input, output)

# Define the twin neural network model
def create_twin_network(input_dim):
    base_network = create_base_network(input_dim)
    
    input_a = Input(shape=(input_dim,))
    input_b = Input(shape=(input_dim,))
    
    processed_a = base_network(input_a)
    processed_b = base_network(input_b)
    
    # Compute the difference between the outputs
    diff = Subtract()([processed_a, processed_b])
    
    return Model([input_a, input_b], diff)

# Custom loss function for semi-supervised learning
def custom_loss_function(lambda_value, b):
    def loss(y_true, y_pred):
        labeled_mask = tf.cast(tf.not_equal(y_true, -1), tf.float32)
        unlabeled_mask = 1.0 - labeled_mask
        
        # Supervised loss for labeled data
        supervised_loss = (2 / b) * tf.reduce_sum(
            tf.square(y_pred - y_true) * labeled_mask
        )
        
        # Unsupervised loss for loop consistency
        unsupervised_loss = (2 * lambda_value / b) * tf.reduce_sum(
            tf.square(y_pred) * unlabeled_mask
        )
        
        return supervised_loss + unsupervised_loss
    
    return loss

# Create the dataset (dummy data for illustration)
num_samples = 100
input_dim = 10
X_train = np.random.rand(num_samples, input_dim)
y_train = np.random.rand(num_samples)

# Create labeled pairs for training
labeled_pairs = []
labeled_labels = []

for i in range(num_samples):
    for j in range(i + 1, num_samples):
        labeled_pairs.append([X_train[i], X_train[j]])
        labeled_labels.append(y_train[i] - y_train[j])

labeled_pairs = np.array(labeled_pairs)
labeled_labels = np.array(labeled_labels)

# Add unlabeled pairs (use -1 as a marker for unlabeled data)
num_unlabeled = 30
unlabeled_data = np.random.rand(num_unlabeled, input_dim)
unlabeled_pairs = []
unlabeled_labels = []

for i in range(num_unlabeled):
    for j in range(i + 1, num_unlabeled):
        unlabeled_pairs.append([unlabeled_data[i], unlabeled_data[j]])
        unlabeled_labels.append(-1)  # -1 indicates unlabeled data

unlabeled_pairs = np.array(unlabeled_pairs)
unlabeled_labels = np.array(unlabeled_labels)

# Add cross pairs between labeled and unlabeled data
cross_pairs = []
cross_labels = []

for i in range(num_samples):
    for j in range(num_unlabeled):
        cross_pairs.append([X_train[i], unlabeled_data[j]])
        cross_labels.append(-1)  # -1 indicates semi-supervised pair

cross_pairs = np.array(cross_pairs)
cross_labels = np.array(cross_labels)

# Combine all pairs
all_pairs = np.concatenate([labeled_pairs, unlabeled_pairs, cross_pairs])
all_labels = np.concatenate([labeled_labels, unlabeled_labels, cross_labels])

# Define and compile the model
twin_network = create_twin_network(input_dim)
twin_network.compile(optimizer=Adam(learning_rate=0.001), loss=custom_loss_function(lambda_value=1.0, b=len(all_pairs)))

# Train the model
twin_network.fit([all_pairs[:, 0], all_pairs[:, 1]], all_labels, epochs=50, batch_size=32)

print("Training completed.")


Epoch 1/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 7.5194e-04
Epoch 2/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3.8706e-04
Epoch 3/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.7699e-04
Epoch 4/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.5300e-04
Epoch 5/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.4702e-04
Epoch 6/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.3373e-04
Epoch 7/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.3332e-04
Epoch 8/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.3369e-04
Epoch 9/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2975e-04
Epoch 10/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.3602e-04
Epoch 11/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2437e-04
Epoch 12/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.2300e-04
Epoch 13/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.2495e-04
Epoch 14/50
263/263 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2.2508e-04
E